In [ ]:
#scraper to take links collected by LibraryThingScraper
#return scrape of website

In [173]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np
import requests
import re

In [210]:
def GetBookPlease(page):
    
    print(page)
        
    #get response
    response = requests.get(url='https://www.librarything.com'+ page).text
    soup = BeautifulSoup(response, features="html.parser")
    
    #get top of page - author, publish date, if part of series
    top = soup.find(attrs={'class':'headsummary'})
    published = top.find(attrs={'class':'date'}).string #hope this always exists
    author = top.find('h2').text                        #hope this always exists
    stuff = top.find_all('h3')                          #this will always exist
    for e in stuff:
        if 'Series:' in e.text:
            series = 1
        else:
            series = 0
    
    #get rating
    rating = soup.find(attrs={'class':'dark_hint'})
    if rating != None:
        rating = soup.find(attrs={'class':'dark_hint'}).text
        
    #get number of characters
    groups = soup.find_all(attrs={'class':'fwikiGroup'})
    for g in range(len(groups)):
        hm = groups[g].find(attrs={'fieldname':'characternames'})
        if hm is not None:
            place = g
    charnum = groups[place].find(attrs={'class':'itemnumberoverflow'})
    if charnum is None:
        charnum = groups[place].find_all(attrs={'class':'fwikiAtomicValue'})
        charnum = len(charnum)
    else:
        charnum = groups[place].find(attrs={'class':'itemnumberoverflow'}).text
        charnum = re.findall(r'\d+', charnum)[0]

    #get number of awards
    #would be nice to have titles but not enough time
    for g in range(len(groups)):
        hm = groups[g].find(attrs={'fieldname':'awards'})
        if hm is not None:
            place = g
    awardnum = groups[place].find(attrs={'class':'itemnumberoverflow'})
    if awardnum is None:
        awardnum = groups[place].find_all(attrs={'class':'fwikiAtomicValue'})
        awardnum = len(awardnum)
    else:
        awardnum = groups[place].find(attrs={'class':'itemnumberoverflow'}).text
        awardnum = re.findall(r'\d+', awardnum)[0]
        
    #print(charnum)
    return [page, published, author, series, rating, charnum, awardnum]


In [211]:
#pull in the link list
moviesdb = pd.read_csv('LTlinksFull.csv')

#make a list of book links
links = moviesdb['Link2']

#print(links.isna().sum())
for m in range(len(links)):
    if str(links[m])=='nan':
        if r']' not in moviesdb.loc[m, 'SearchTerm']:
            #print(moviesdb.loc[m, 'Link2'])
            links[m] = moviesdb.loc[m, 'Link1']

#print(links[10:20])

In [212]:
library = pd.DataFrame(columns = ['link','published', 'author', 'series', 'rating', 'charnum', 'awardnum'])
#links = ['/work/4041453', '/work/1906740', '/work/4725', '/work/5716']

for l in range(len(links)):
    if str(links[l]) != 'nan':
        grab = GetBookPlease(links[l])
        library = library.append(pd.Series(grab, index = library.columns), ignore_index=True)

print(library)

/work/21214
/work/364
/work/24434168
/work/21623
/work/831523
/work/23961085
/work/6664405


AttributeError: 'NoneType' object has no attribute 'string'

In [ ]:
#starting pull at 3:17